In [1]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling dde5aa3fc5ff: 100% ▕██████████████████▏ 2.0 GB                         
pulling 966de95ca8a6: 100% ▕██████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da: 100% ▕██████████████████▏ 7.7 KB                         
pulling a70ff7e570d9: 100% ▕██████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5: 100% ▕██████████████████▏   96 B                         
pulling 34bb5ab01051: 100% ▕██████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [3]:
MODEL = "llama3.2"
openai = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [4]:
class Website:
    """
    A utility class to represent a website that we have scraped
    """
    url: str
    title: str
    text: str

    def __init__(self, url):
        """
        Create this website object from the given url using the beautiful library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(['script', 'style', 'img', 'input']):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator='\n', strip=True)
        

In [5]:
eg = Website("https://www.betterup.com/blog/activities-for-personality-development")
print(eg.title)
print(eg.text)

15 Personality Development Activities for Personal Growth
Search
Toggle Search
EN - US
English US
Deutsch
English GB
Français
For Business
Platform
Platform overview
Integrations
Powered by AI
Products
BetterUp Lead™
BetterUp Manage™
BetterUp Ready™
BetterUp Grow™
Solutions
Sales Performance
Executive
Government
Customers
Case studies
ROI of BetterUp
Resources
Research
Content library
Courageous leadership development hub
Center for Daring Leadership
Built for growth in the age of AI:
Explore on-demand sessions, breakthrough research, and powerful takeaways from Uplift 2025
Learn more
Learn more
For Individuals
What is Coaching?
About Coaching
Find your Coach
Types of Coaching
Career Coaching
Communications Coaching
Personal Coaching
Resources
Blog
Discover your perfect match
: Take our 5-minute assessment and let us pair you with one of our top Coaches tailored just for you.
Find your coach
Find your coach
About
About Us
We're on a mission to help everyone live with clarity, purpose, 

## Types of prompts
You may know this already- but if not get very familiar with it!

Models like GPT4o have been trained to recieve instruction in a particular way.

They expect to recieve:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** the conversation starter that they should reply to

In [6]:
# Define our system prompt
sys_prompt = "Yor are an assistant that  analyzes the contents of a website and provides a short summary, ignoring the text that might be navigation related. Respond in markdown."

In [7]:
def user_prompt_for(website):
    user_prompt = f"you looking at website titled {website.title}"
    user_prompt += "The content of this website is as follows; please provide a short summary of this website in markdown. If it includes new or announcements, then summarize these too."
    user_prompt += website.text
    return user_prompt

In [8]:
print(user_prompt_for(eg))

you looking at website titled 15 Personality Development Activities for Personal GrowthThe content of this website is as follows; please provide a short summary of this website in markdown. If it includes new or announcements, then summarize these too.Search
Toggle Search
EN - US
English US
Deutsch
English GB
Français
For Business
Platform
Platform overview
Integrations
Powered by AI
Products
BetterUp Lead™
BetterUp Manage™
BetterUp Ready™
BetterUp Grow™
Solutions
Sales Performance
Executive
Government
Customers
Case studies
ROI of BetterUp
Resources
Research
Content library
Courageous leadership development hub
Center for Daring Leadership
Built for growth in the age of AI:
Explore on-demand sessions, breakthrough research, and powerful takeaways from Uplift 2025
Learn more
Learn more
For Individuals
What is Coaching?
About Coaching
Find your Coach
Types of Coaching
Career Coaching
Communications Coaching
Personal Coaching
Resources
Blog
Discover your perfect match
: Take our 5-minute

# Messages
The API from OpenAI expects to receive messages in a particular structure. Many of the other APIs share this structure:

[

    {"role": "system", "content": "system message goes here"},
    
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [9]:
# See how this function creates exactly the format above
def messages_for(website):
    return [
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [10]:
messages_for(eg)

[{'role': 'system',
  'content': 'Yor are an assistant that  analyzes the contents of a website and provides a short summary, ignoring the text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "you looking at website titled 15 Personality Development Activities for Personal GrowthThe content of this website is as follows; please provide a short summary of this website in markdown. If it includes new or announcements, then summarize these too.Search\nToggle Search\nEN - US\nEnglish US\nDeutsch\nEnglish GB\nFrançais\nFor Business\nPlatform\nPlatform overview\nIntegrations\nPowered by AI\nProducts\nBetterUp Lead™\nBetterUp Manage™\nBetterUp Ready™\nBetterUp Grow™\nSolutions\nSales Performance\nExecutive\nGovernment\nCustomers\nCase studies\nROI of BetterUp\nResources\nResearch\nContent library\nCourageous leadership development hub\nCenter for Daring Leadership\nBuilt for growth in the age of AI:\nExplore on-demand sessions, breakthrough research, a

# Time to bring it together- The API for the OpenAI is very simple!

In [11]:
# And now call the openai API. You will get very familiar with this!
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [12]:
print(summarize("https://www.betterup.com/blog/activities-for-personality-development"))

**Personal Development Activities to Help You Become the Best Version of Yourself**

Personal development is an ongoing journey that requires effort, patience, and dedication. The following list of activities can help you take control of your personal growth and become a better version of yourself.

### 1. **Step Out of Your Comfort Zone**
Challenge yourself by trying new things and taking calculated risks to gain experience and build confidence.

### 2. **Find a Mentor/Coach**
Work with a professional coach or mentor to receive guidance, support, and accountability on your personal development journey.

### 3. **Volunteer for a Cause You're Passionate About**
Help others while developing new skills and networking opportunities that align with your values.

### 4. **Get Feedback from Others**
Ask trusted individuals who know you well to provide honest feedback and insights into areas where you can improve.

### 5. **Practice Mindfulness and Self-Care**
Regular mindfulness practice will

In [15]:
# A function to display this nicely in the jupyter output, using markdown
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [16]:
display_summary("https://www.betterup.com/blog/activities-for-personality-development")

# Activities for Personality Development

Many individuals struggle with personal development due to the complexity of personal change and the difficulty in driving progress on their own.

## Key Concepts
In this post, we will briefly touch upon key concepts which are also relevant to our discussion:
*   *The Importance of Self-Awareness*: Understanding our strengths, weaknesses, values, and goals is essential for making positive changes in our lives.
*   *Building Confidence*: Developing self-confidence takes time and practice. It's possible by doing activities that promote confidence and overcoming challenges.
*   *Developing Emotional Intelligence*: Practicing empathy and understanding ourselves better will help us to become more empathetic towards others.

## Activities for Personality Development

The following list offers advice on how to drive progress in your personal development:

**Key Activities:**

\.\. \- Say no with intention
\.\.\- Seek out opportunities of improvement
\.\-\ Set realistic goals, tracking them will provide a sense of fulfillment.
\._- Learn from failures and challenges by maintaining the practice.